In [ ]:
#get all the submissions from the day 
#get 10 with most twitter citations
#summarize and post them 

In [3]:
"""
based on ET - every day one hour past 00:00 past ET i will tweet it 

go to certain website 

1. get date of day before 
2. open first page and check if date is equals to target date if yes add all the paper urls from the page to the list and if date is older then target date exit
3. open all pages, get the number of tweet impressions, get the url to the pdf, title and subject area
4. make summary and post
"""
from datetime import datetime, timedelta
import pytz
import requests
from bs4 import BeautifulSoup
import time

# Get date from yesterday ET time
eastern = pytz.timezone('US/Eastern')
yesterday = datetime.now(eastern) - timedelta(days=1)
yesterday_date = yesterday.strftime('%Y-%m-%d')
print("Yesterday's date in ET:", yesterday_date)

page_data = []

def iterate_biorxiv_pages():
    page_number = 0
    while True:
        url = f"https://www.biorxiv.org/content/early/recent?page={page_number}"
        print(url)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Get the date based on the X path
        date_element = soup.select_one('#block-system-main > div > div > div > div:nth-child(2) > div:nth-child(1) > div > div > div > div > div:nth-child(1) > h3')
        date_text = date_element.get_text(strip=True)
        
        # Reformat the date to match the format of our yesterday_date
        page_date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')

        print("page date: ", page_date)
        
        # Check if the date is older than the target date
        if page_date < yesterday_date:
            break
        
        # Skip the page if the date is newer than the target date
        if page_date > yesterday_date:
            page_number += 1
            continue

        # Extract all the URLs from the page
        article_blocks = soup.find_all('div', class_='highwire-cite highwire-cite-highwire-article highwire-citation-biorxiv-article-pap-list-overline clearfix')
        
        for block in article_blocks:
            title_element = block.find('span', class_='highwire-cite-title')
            if title_element:
                link_element = title_element.find('a', class_='highwire-cite-linked-title')
                if link_element and 'href' in link_element.attrs:
                    paper_url = "https://www.biorxiv.org" + link_element['href']
                    page_data.append(paper_url)
        
        #here i can extract all the data

        # If date is target date, get all the data and save it in list
        print(page_date)
        # Add logic to extract and save data here
        
        page_number += 1
    return page_data

page_data = iterate_biorxiv_pages()

Yesterday's date in ET: 2024-08-30
https://www.biorxiv.org/content/early/recent?page=0
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=1
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=2
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=3
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=4
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=5
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=6
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=7
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=8
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=9
page date:  2024-08-30
2024-08-30
https://www.biorxiv.org/content/early/recent?page=10
page date:  2024-08-30
2024-08-30
https://www.biorxi

In [ ]:
#i 

In [ ]:
import asyncio
from playwright.async_api import async_playwright, TimeoutError
import nest_asyncio
from bs4 import BeautifulSoup
from datetime import date, timedelta

# Function to generate the Hugging Face URL
def generate_huggingface_url():
    yesterday = date.today() - timedelta(days=1)
    url = f"https://huggingface.co/papers?date={yesterday.year}-{yesterday.month:02d}-{yesterday.day:02d}"
    return url

# Async function to fetch the HTML content
async def fetch_papers():
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        page = await browser.new_page()
        huggingface_url = generate_huggingface_url()
        await page.goto(huggingface_url)
        html_content = await page.content()
        papers = parse_html_content(html_content)
        return papers

# Function to parse the HTML content and extract paper details
def parse_html_content(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    papers = []
    for container in soup.find_all('div', class_='from-gray-50-to-white -mt-2 flex bg-gradient-to-b px-6 pb-6 pt-8'):
        link = container.find('a', class_='line-clamp-3 cursor-pointer text-balance')
        if link:
            href = link.get('href')
            arxiv_url = f"https://arxiv.org/pdf{href[href.rfind('/'):]}.pdf"
            paper_name = link.text.strip()
            upvotes_div = container.find('div', class_='shadow-alternate').find('div', class_='leading-none')
            if upvotes_div:
                upvotes = upvotes_div.text.strip()
                papers.append((arxiv_url, paper_name, upvotes))
        else:
            print("Link not found in this container.")
    return papers

# Ensure nest_asyncio is applied
nest_asyncio.apply()